In [ ]:
import pandas as pd
import numpy as np
import copy

In [ ]:
ins = pd.read_excel("xlsx_file", sheet_name=0, index_col=0)
uri = pd.read_excel("xlsx_file", sheet_name=1, index_col=0)


In [ ]:
reins = ins.drop(["", ""], axis=1)
reins = reins.fillna(0).replace("○", 1).replace("", 0).replace("\u3000", 0).replace("-", 0)

reuri = uri.iloc[:, 0:9]
reuri = reuri.drop(["", ""], axis=1)

reins, reuri = reins.to_numpy().tolist(), reuri.to_numpy().tolist()


In [ ]:
reins = list(map(list, set(map(tuple, reins))))
reuri = list(map(list, set(map(tuple, reuri))))


In [ ]:
Input = []

for i in range(len(reins)):
    for j in range(len(reuri)):
        if reins[i][1] == reuri[j][0] and reins[i][3] == reuri[j][1]:
            reins[i].extend(reuri[j][2:])
            Input.append(reins[i])

Input = [i[0:20] for i in Input]


In [ ]:
# 必要なデータから0を抜く
hlin0 = []
for i in range(len(Input)):
    if Input[i][18] != 0:
        hlin0.append(Input[i])

hlin00 = []
for i in range(len(hlin0)):
    if hlin0[i][19] != 0:
        hlin00.append(hlin0[i])

hlin0 = []
for i in range(len(hlin00)):
    if hlin00[i][2] != "サービス業(他に分類されないもの)":
        hlin0.append(hlin00[i])

hlin00 = []
for i in range(len(hlin0)):
    if hlin0[i][2] != "複合サービス事業":
        hlin00.append(hlin0[i])

hlin0 = hlin00


In [ ]:
for i in range(len(hlin0)):
    if hlin0[i][2] == "不動産業，物品賃貸業":
        hlin0[i][2] = -0.0773
    elif hlin0[i][2] == "建設業":
        hlin0[i][2] = -1.4450
    elif hlin0[i][2] == "情報通信業":
        hlin0[i][2] = -0.1350
    elif hlin0[i][2] == "林業":
        hlin0[i][2] = -0.4030
    elif hlin0[i][2] == "電気・ガス・熱供給・水道業":
        hlin0[i][2] = -0.9330
    elif hlin0[i][2] == "生活関連サービス業，娯楽業":
        hlin0[i][2] = -1.004
    elif hlin0[i][2] == "卸売業，小売業":
        hlin0[i][2] = -0.4550
    elif hlin0[i][2] == "医療，福祉":
        hlin0[i][2] = -0.6319
    elif hlin0[i][2] == "宿泊業，飲食サービス業":
        hlin0[i][2] = -0.4607
    elif hlin0[i][2] == "製造業":
        hlin0[i][2] = -0.7577
    elif hlin0[i][2] == "教育，学習支援業":
        hlin0[i][2] = -0.0654
    elif hlin0[i][2] == "学術研究，専門・技術サービス業":
        hlin0[i][2] = -0.1173
    elif hlin0[i][2] == "金融業，保険業":
        hlin0[i][2] = -1.7570
    elif hlin0[i][2] == "運輸業，郵便業":
        hlin0[i][2] = -0.8893


In [ ]:
for i in range(len(hlin0)):
    if hlin0[i][4] == "内部犯罪・内部不正行為":
        hlin0[i][4] = 1
    elif hlin0[i][4] == "不正な情報持ち出し":
        hlin0[i][4] = 1
    else:
        hlin0[i][4] = 0


In [ ]:
import requests
from bs4 import BeautifulSoup


In [ ]:
url = "http://windy.mind.meiji.ac.jp/~atsu110/Yamada/index.php"
result = []

for i in range(len(hlin0)):
    p = {"people": hlin0[i][3], "value": hlin0[i][18], "intent": hlin0[i][4], "res": hlin0[i][16], "eco": hlin0[i][13], "mind": hlin0[i][14], "indus": hlin0[i][2], "name": hlin0[i][5],
         "adress": hlin0[i][6], "number": hlin0[i][7], "date": hlin0[i][8], "sex": hlin0[i][9], "job": hlin0[i][10], "email": hlin0[i][11], "id": hlin0[i][12], "mul": 0, "law": 0, "soc": 1}
    hdoc = requests.get(url, params=p).content
    soup = BeautifulSoup(hdoc, "html.parser")
    price = soup.find_all("td")
    result.append([hlin0[i][0], hlin0[i][1], hlin0[i][19], str(price[1].contents[0]).split()[
                  0], str(price[3].contents[0]).split()[0], str(price[5].contents[0]).split()[0]])


In [ ]:
x = []
y = []
z = []
w = []
for i in range(len(result)):
    x.append(float(result[i][2]))
    y.append(float(result[i][3].replace(",", ""))/100)
    z.append(float(result[i][4].replace(",", ""))/100)
    w.append(float(result[i][5].replace(",", ""))*133/100)


In [ ]:
for i in range(len(result)):
    result[i][0] = result[i][0].year
    result[i][2] = x[i]
    result[i][3] = y[i]
    result[i][4] = z[i]
    result[i][5] = w[i]


In [ ]:
result01 = [result[i][1:3] for i in range(len(result))]
result02 = list(map(list, set(map(tuple, result01))))
[result02[i].extend([0, 0, 0]) for i in range(len(result02))]
for i in range(len(result02)):
    for j in range(len(result)):
        if result02[i][0:2] == result[j][1:3]:
            result02[i][2] += result[j][3]
            result02[i][3] += result[j][4]
            result02[i][4] += result[j][5]


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

x, y = [], []
for i in range(len(result02)):
    x.append(result02[i][1])
    y.append(result02[i][2])

r2 = r2_score(x, y)
rmse = np.sqrt(mean_squared_error(x, y))
mae = mean_absolute_error(x, y)

print(r2, rmse, mae)


plt.scatter(x, y)
plt.plot([0, 600000], [0, 600000])
plt.show()


In [ ]:
x, y = [], []
for i in range(len(result02)):
    x.append(result02[i][1])
    y.append(result02[i][3])

r2 = r2_score(x, y)
rmse = np.sqrt(mean_squared_error(x, y))
mae = mean_absolute_error(x, y)

print(r2, rmse, mae)


plt.scatter(x, y)
plt.plot([0, 400000], [0, 400000])
plt.show()


In [ ]:
x, y = [], []
for i in range(len(result02)):
    x.append(result02[i][1])
    y.append(result02[i][4])

r2 = r2_score(x, y)
rmse = np.sqrt(mean_squared_error(x, y))
mae = mean_absolute_error(x, y)

print(r2, rmse, mae)


plt.scatter(x, y)
plt.plot([0, 400000], [0, 400000])
plt.show()


In [ ]:
xm = 100000
xM = 0
im = 0
iM = 0
for i in result02:
    x1 = abs(i[1] - i[2])
    if xm > x1:
        xm = x1
        jm = i
    if x1 > xM:
        xM = x1
        jM = i
print(jm, xm)
print(jM, xM)


In [ ]:
import statsmodels.api as sm
import math

hlinM = copy.deepcopy(hlin0)

for i in range(len(hlinM)):
    hlinM[i][3] = math.log(float(hlinM[i][3]), math.e)
    hlinM[i][18] = math.log(float(hlinM[i][18]), math.e)
    hlinM[i][19] = math.log(float(hlinM[i][19]), math.e)

kai = pd.DataFrame(hlinM)
kx = kai[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]
ky = kai[[19]]

kx = sm.add_constant(kx)

OLS = sm.OLS(ky, kx).fit()
GLS = sm.GLS(ky, kx).fit()
GLSAR = sm.GLSAR(ky, kx).fit()
WLS = sm.WLS(ky, kx).fit()
RecursiveLS = sm.RecursiveLS(ky, kx).fit()
# RollingOLS = sm.RollingOLS(ky, kx).fit()
# RollingWLS = sm.RollingWLS(ky, kx).fit()


In [ ]:
OLS.summary()
